In [18]:
import xarray as xr
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import fsspec


In [1]:
# Common imports and settings
import os, sys, re
from pathlib import Path
from IPython.display import Markdown
import pandas as pd
pd.set_option("display.max_rows", None)
import xarray as xr
import dask
from dask.distributed import Client
from dask_gateway import Gateway
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta

# Datacube
import datacube
from datacube.utils.aws import configure_s3_access
import odc.geo.xr                                  # https://github.com/opendatacube/odc-geo
from datacube.utils import masking  # https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/masking.py
from odc.algo import enum_to_bool                  # https://github.com/opendatacube/odc-tools/blob/develop/libs/algo/odc/algo/_masking.py
from dea_tools.plotting import display_map, rgb    # https://github.com/GeoscienceAustralia/dea-notebooks/tree/develop/Tools

import boto3

# Basic plots
%matplotlib inline
# import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize'] = [12, 8]

# Holoviews
# https://holoviz.org/tutorial/Composing_Plots.html
# https://holoviews.org/user_guide/Composing_Elements.html
import hvplot.pandas
import hvplot.xarray
import panel as pn
import colorcet as cc
import cartopy.crs as ccrs
from datashader import reductions
from holoviews import opts
# hv.extension('bokeh', logo=False)
print("Libraries loaded successfully.")

Libraries loaded successfully.


In [2]:
# EASI defaults
# These are convenience functions so that the notebooks in this repository work in all EASI deployments

# The `git.Repo()` part returns the local directory that easi-notebooks has been cloned into
# If using the `easi-tools` functions from another path, replace `repo` with your local path to `easi-notebooks` directory
try:
    import git
    repo = git.Repo('.', search_parent_directories=True).working_tree_dir    # Path to this cloned local directory
except (ImportError, git.InvalidGitRepositoryError):
    repo = Path.home() / 'easi-notebooks'    # Reasonable default
    if not repo.is_dir():
        raise RuntimeError('To use `easi-tools` please provide the local path to `https://github.com/csiro-easi/easi-notebooks`')
if repo not in sys.path:
    sys.path.append(str(repo))    # Add the local path to `easi-notebooks` to python

from easi_tools import EasiDefaults
from easi_tools import initialize_dask, xarray_object_size, mostcommon_crs, heading
#from easi_tools.load_s2l2a import load_s2l2a_with_offset
print("EASI librariies loaded successfully.")

EASI librariies loaded successfully.


In [3]:
gateway = Gateway()
options = gateway.cluster_options()
options.cuda_worker = True
clusters = gateway.list_clusters()
if not clusters:
    print('Creating new cluster...')
    cluster = gateway.new_cluster(cluster_options=options)
else:
    print(f'Connecting to existing cluster: {clusters[0].name}')
    cluster = gateway.connect(clusters[0].name)

NUM_WORKERS = 4
cluster.scale(NUM_WORKERS)
client = cluster.get_client()

print("Cluster Dashboard:", client.dashboard_link)


Creating new cluster...


KeyboardInterrupt: 

In [21]:
# --- Configuration (Must match your saving path) ---
easi = EasiDefaults()
bucket = easi.scratch

# 2. Get User ID
userid = boto3.client('sts').get_caller_identity()['UserId']

# 3. Define your Project Name and Dataset Name
project_name = "traning_test_project"
dataset_name = "training_dataset_v1.zarr"

# 4. Construct the full S3 Path
S3_OUTPUT_PATH = f"s3://{bucket}/{userid}/{project_name}/{dataset_name}"

print(f"Target Storage Path: {S3_OUTPUT_PATH}")
PATCH_SIZE = 224 # Must match the chunk size used during Dask preprocessing

# --- 1. Load the Zarr store lazily ---
configure_s3_access(aws_unsigned=False, requester_pays=True, client=client)
# Use fsspec to map the S3 path, which allows xarray to read chunk-by-chunk
try:
    s3_mapper = fsspec.get_mapper(S3_OUTPUT_PATH)
    # Open the Zarr store. This is a lazy operation.
    training_ds_read = xr.open_zarr(s3_mapper, consolidated=True)
    print("Zarr Dataset successfully opened from S3.")
except Exception as e:
    print(f"Error opening Zarr store: {e}")
    # You might need to adjust storage_options depending on your S3 configuration
    print("Please verify S3_OUTPUT_PATH and AWS access/permissions.")

training_ds_read

Successfully found configuration for deployment "csiro"
Target Storage Path: s3://easihub-csiro-user-scratch/AROAWO7MSC2T2SINGSYOQ:csiro-csiro-aad_gre555@csiro.au/traning_test_project/training_dataset_v1.zarr
Zarr Dataset successfully opened from S3.


<xarray.Dataset> Size: 48GB
Dimensions:      (band: 6, time: 43, y: 6441, x: 5469)
Coordinates:
  * band         (band) object 48B 'nbart_blue' 'nbart_green' ... 'nbart_swir_2'
    spatial_ref  int32 4B ...
  * time         (time) datetime64[ns] 344B 2020-02-08T23:56:31.774346 ... 20...
  * x            (x) float64 44kB 1.272e+06 1.272e+06 ... 1.436e+06 1.436e+06
  * y            (y) float64 52kB -3.922e+06 -3.922e+06 ... -4.115e+06
Data variables:
    features     (band, time, y, x) float32 36GB dask.array<chunksize=(1, 1, 224, 224), meta=np.ndarray>
    labels       (time, y, x) int64 12GB dask.array<chunksize=(1, 224, 224), meta=np.ndarray>

In [22]:
# distributed_dataset.py
"""
Dataset and training utilities that run entirely on Dask workers.
Nothing runs on the local notebook except job submission.
"""

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import GradScaler, autocast
import xarray as xr
import fsspec
import numpy as np
from typing import Dict, List, Tuple, Optional
import io


class RemoteZarrDataset(Dataset):
    """
    Dataset that initializes completely on remote workers.
    No S3 calls happen on the client/notebook.
    """
    
    def __init__(
        self,
        zarr_path: str,
        patch_size: int = 224,
        worker_rank: int = 0,
        world_size: int = 1,
        feature_var: str = 'features',
        label_var: str = 'labels',
        num_threads: int = 8,
    ):
        self.zarr_path = zarr_path
        self.patch_size = patch_size
        self.worker_rank = worker_rank
        self.world_size = world_size
        self.feature_var = feature_var
        self.label_var = label_var
        # Might not use but whatevre
        self.num_threads = num_threads
        
        # These are set lazily on first access
        self._ds = None
        self._indices = None
        self._metadata = None
    
    def _ensure_initialized(self):
        """Initialize on first access (runs on worker, not client)."""
        if self._ds is not None:
            return
        
        # Open connection
        mapper = fsspec.get_mapper(self.zarr_path)
        self._ds = xr.open_zarr(mapper, consolidated=True)
        
        # Get metadata
        features = self._ds[self.feature_var]
        chunks = features.chunks
        
        time_dim = features.get_axis_num('time')
        y_dim    = features.get_axis_num('y')
        x_dim    = features.get_axis_num('x')
        
        time_chunks = chunks[time_dim]
        y_chunks    = chunks[y_dim]
        x_chunks    = chunks[x_dim]
        
        n_time = len(time_chunks)   # number of time *chunks* (with your config = number of time steps)
        n_y    = len(y_chunks)      # number of y chunks
        n_x    = len(x_chunks)      # number of x chunks
        
        self._metadata = {
            'n_time': n_time,
            'n_y': n_y,
            'n_x': n_x,
            'total_patches': n_time * n_y * n_x,
            'dims': dict(features.sizes),
        }
        
        # Compute this worker's indices
        total = self._metadata['total_patches']
        per_worker = total // self.world_size
        start = self.worker_rank * per_worker
        end = start + per_worker if self.worker_rank < self.world_size - 1 else total
        self._indices = list(range(start, end))
    
    def __len__(self) -> int:
        self._ensure_initialized()
        return len(self._indices)
    
    def _global_to_coords(self, global_idx: int) -> Tuple[int, int, int]:
        """Convert global index to (time, y, x) chunk coordinates."""
        m = self._metadata
        patches_per_time = m['n_x'] * m['n_y']
        
        t = global_idx // patches_per_time
        remainder = global_idx % patches_per_time
        y = remainder // m['n_x']
        x = remainder % m['n_x']
        
        return t, y, x
    
    # def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
    #     self._ensure_initialized()
        
    #     # Map local index to global index
    #     global_idx = self._indices[idx]
    #     t, y_chunk, x_chunk = self._global_to_coords(global_idx)
        
    #     # Pixel bounds
    #     y_start = y_chunk * self.patch_size
    #     x_start = x_chunk * self.patch_size
    #     y_end = min(y_start + self.patch_size, self._metadata['dims']['y'])
    #     x_end = min(x_start + self.patch_size, self._metadata['dims']['x'])
        
    #     # Load
    #     features = self._ds[self.feature_var].isel(
    #         time=t, y=slice(y_start, y_end), x=slice(x_start, x_end)
    #     ).values
        
    #     labels = self._ds[self.label_var].isel(
    #         time=t, y=slice(y_start, y_end), x=slice(x_start, x_end)
    #     ).values
        
    #     # Pad if needed
    #     if features.shape[-2:] != (self.patch_size, self.patch_size):
    #         features = self._pad(features, 3)
    #         labels = self._pad(labels, 2)
        
    #     return (
    #         torch.from_numpy(features).float(),
    #         torch.from_numpy(labels).long()
    #     )

    def __getitem__(self, idx: int):
        self._ensure_initialized()
        
        global_idx = self._indices[idx]
        t, y_chunk, x_chunk = self._global_to_coords(global_idx)
        
        y_start = y_chunk * self.patch_size
        x_start = x_chunk * self.patch_size
        y_end = min(y_start + self.patch_size, self._metadata['dims']['y'])
        x_end = min(x_start + self.patch_size, self._metadata['dims']['x'])
        
        # Use a local thread scheduler, not the distributed client
        with dask.config.set(scheduler='threads'):
            features = self._ds[self.feature_var].isel(
                time=t, y=slice(y_start, y_end), x=slice(x_start, x_end)
            ).values
        
            labels = self._ds[self.label_var].isel(
                time=t, y=slice(y_start, y_end), x=slice(x_start, x_end)
            ).values
        
        if features.shape[-2:] != (self.patch_size, self.patch_size):
            features = self._pad(features, 3)
            labels = self._pad(labels, 2)
        
        return (
            torch.from_numpy(features).float(),
            torch.from_numpy(labels).long()
        )
        
    def _pad(self, arr, ndim):
        h, w = arr.shape[-2], arr.shape[-1]
        pad_h, pad_w = self.patch_size - h, self.patch_size - w
        if ndim == 3:
            return np.pad(arr, ((0,0), (0,pad_h), (0,pad_w)))
        return np.pad(arr, ((0,pad_h), (0,pad_w)))
    
    def get_batch(self, indices: list) -> tuple:
        """
        Fetch multiple items concurrently using threads.
        Call this instead of iterating with DataLoader.
        """
        from concurrent.futures import ThreadPoolExecutor
        import torch
        
        with ThreadPoolExecutor(max_workers=min(len(indices), 8)) as executor:
            results = list(executor.map(self.__getitem__, indices))
        
        features = torch.stack([r[0] for r in results])
        labels = torch.stack([r[1] for r in results])
        
        return features, labels
    
    def iter_batches(self, batch_size: int, shuffle: bool = True):
        """
        Generator that yields batches with concurrent fetching.
        Use this instead of DataLoader inside Dask workers.
        """
        import random
        
        indices = list(range(len(self)))
        if shuffle:
            random.shuffle(indices)
        
        for start in range(0, len(indices), batch_size):
            batch_indices = indices[start:start + batch_size]
            yield self.get_batch(batch_indices)
    
    def _ensure_initialized(self):
        if self._ds is not None:
            return

        mapper = fsspec.get_mapper(self.zarr_path)
        self._ds = xr.open_zarr(mapper, consolidated=True)

        features = self._ds[self.feature_var]  # dims: ('band','time','y','x')
        chunks = features.chunks               # tuple of chunk sequences per dim

        time_dim = features.get_axis_num('time')
        y_dim    = features.get_axis_num('y')
        x_dim    = features.get_axis_num('x')

        time_chunks = chunks[time_dim]
        y_chunks    = chunks[y_dim]
        x_chunks    = chunks[x_dim]

        # Sanity: we assume fixed chunk sizes in y/x
        y_chunk_size = y_chunks[0]
        x_chunk_size = x_chunks[0]

        if self.patch_size != y_chunk_size or self.patch_size != x_chunk_size:
            raise ValueError(
                f"patch_size={self.patch_size} must equal chunk size "
                f"({y_chunk_size}, {x_chunk_size}) for current indexing logic"
            )

        n_time = len(time_chunks)
        n_y    = len(y_chunks)
        n_x    = len(x_chunks)

        self._metadata = {
            'n_time': n_time,
            'n_y': n_y,
            'n_x': n_x,
            'total_patches': n_time * n_y * n_x,
            'dims': dict(features.sizes),
        }

        # Shard indices by (rank, world_size)
        total = self._metadata['total_patches']
        per_worker = total // self.world_size
        start = self.worker_rank * per_worker
        end = start + per_worker if self.worker_rank < self.world_size - 1 else total
        self._indices = list(range(start, end))

    def xr_ds(self):
        """Public accessor for the underlying xarray.Dataset."""
        self._ensure_initialized()
        return self._ds

# Test it
def test_builtin_batching(zarr_path: str, batch_size: int = 4):
    import time
    
    ds = RemoteZarrDataset(zarr_path=zarr_path, patch_size=PATCH_SIZE, worker_rank=0, world_size=4)
    
    start = time.time()
    for i, (feat, lab) in enumerate(ds.iter_batches(batch_size)):
        if i >= 3:
            break
    elapsed = time.time() - start
    
    return {
        'batch_shape': tuple(feat.shape),
        'patches_per_second': (3 * batch_size) / elapsed,
    }

result = client.submit(test_builtin_batching, S3_OUTPUT_PATH, batch_size=4).result()
print(f"✓ Built-in batching: {result['patches_per_second']:.1f} patches/sec")

✓ Built-in batching: 3.8 patches/sec


# Various testing and implimentations of different dataloaders
- Settled on Built-in Threading to dataloader as that gave the highest consistant throughput on testing

In [23]:
# --- This code runs on the Dask Worker ---

def test_builtin_batching(
    zarr_path: str, 
    batch_size: int = 8, 
    rank: int = 0, 
    world_size: int = 1, 
    num_threads: int = 16
):
    import time
    
    # ⚠️ NOTE: RemoteZarrDataset is assumed to be defined in the worker environment
    # Pass all dynamic arguments to ensure sharding and threading are used
    ds = RemoteZarrDataset(
        zarr_path=zarr_path, 
        patch_size=PATCH_SIZE, 
        worker_rank=rank, 
        world_size=world_size,
        num_threads=num_threads # Use the thread count we tuned
    )
    
    start = time.time()
    # Read a significant number of batches for stable throughput measurement
    num_batches_to_test = 20 
    
    for i, (feat, lab) in enumerate(ds.iter_batches(batch_size)):
        # Check shape on the first batch read to verify patch_size
        if i == 0:
            print(f"[Rank {rank}] Initializing. Worker reading {len(ds)} patches.")
            print(f"[Rank {rank}] First batch shape: {feat.shape}")
            
        # Optional: Add a simple computation to simulate the GPU being busy
        # feat.mean() 
        
        if i >= num_batches_to_test - 1:
            break
            
    elapsed = time.time() - start
    
    return {
        'rank': rank,
        'batches_read': num_batches_to_test,
        'total_patches': num_batches_to_test * batch_size,
        'patches_per_second': (num_batches_to_test * batch_size) / elapsed,
    }

# --- This code runs on the Dask Client / Notebook ---

# --- Configuration ---
BATCH_SIZE = 8
NUM_THREADS = 16 # Use the best thread count you determined

# 1. Get Cluster Info
workers = client.scheduler_info()['workers']
world_size = len(workers)
futures = []

if world_size < 1:
    raise RuntimeError("No Dask workers found. Please scale up your cluster.")

print(f"Simulating full training conditions across {world_size} concurrent workers.")

# 2. Submit Parallel Tests
for rank, worker_address in enumerate(workers):
    worker_key = worker_address
    
    # Submit the test function, passing the explicit sharding and performance args
    future = client.submit(
        test_builtin_batching, 
        zarr_path=S3_OUTPUT_PATH,
        batch_size=BATCH_SIZE,
        rank=rank,
        world_size=world_size,
        num_threads=NUM_THREADS,
        workers=worker_key # Pin the task
    )
    futures.append(future)

# 3. Gather Results and Verify
results = client.gather(futures)

print("\n--- Test Verification Results ---")
total_throughput = 0
for res in results:
    total_throughput += res['patches_per_second']
    print(f"✅ Rank {res['rank']} (Shard {res['rank'] + 1}/{world_size}): {res['patches_per_second']:.1f} patches/sec")
    
print(f"\n⚡ AGGREGATE THROUGHPUT (Total Cluster I/O): {total_throughput:.1f} patches/sec")

Simulating full training conditions across 4 concurrent workers.

--- Test Verification Results ---
✅ Rank 0 (Shard 1/4): 5.3 patches/sec
✅ Rank 1 (Shard 2/4): 5.1 patches/sec
✅ Rank 2 (Shard 3/4): 5.0 patches/sec
✅ Rank 3 (Shard 4/4): 5.3 patches/sec

⚡ AGGREGATE THROUGHPUT (Total Cluster I/O): 20.8 patches/sec


In [98]:
# Quick test - runs entirely on a Dask worker
def test_dataset_on_worker(zarr_path: str, num_samples: int = 3):
    """Test dataset on remote worker - returns summary info."""
    import torch
    
    # Create dataset
    ds = RemoteZarrDataset(
        zarr_path=zarr_path,
        patch_size=PATCH_SIZE,
        worker_rank=0,
        world_size=1,
    )
    
    # Test length
    length = len(ds)
    
    # Test fetching samples
    samples = []
    for i in range(min(num_samples, length)):
        features, labels = ds[i]
        samples.append({
            'idx': i,
            'features_shape': tuple(features.shape),
            'labels_shape': tuple(labels.shape),
            'features_dtype': str(features.dtype),
            'features_range': (float(features.min()), float(features.max())),
            'unique_labels': torch.unique(labels).tolist()[:10],  # First 10
        })
    
    return {
        'status': 'success',
        'total_samples': length,
        'metadata': ds._metadata,
        'sample_tests': samples,
    }

# Run test on a worker (not locally!)
future = client.submit(test_dataset_on_worker, S3_OUTPUT_PATH)

def test_dataloader_on_worker(zarr_path: str, batch_size: int = 4):
    """Test full DataLoader pipeline on worker."""
    from torch.utils.data import DataLoader
    import time
    
    ds = RemoteZarrDataset(zarr_path, PATCH_SIZE, 0, 1)
    
    # num_workers=0 required inside Dask workers (daemon process limitation)
    loader = DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=0)
    
    start = time.time()
    for i, (feat, lab) in enumerate(loader):
        if i >= 3:
            break
    elapsed = time.time() - start
    
    return {
        'batches_tested': 3,
        'batch_shape': tuple(feat.shape),
        'time_seconds': elapsed,
        'patches_per_second': (3 * batch_size) / elapsed,
    }

result = client.submit(test_dataloader_on_worker, S3_OUTPUT_PATH).result()
print(f"✓ DataLoader works! {result['patches_per_second']:.1f} patches/sec")
result = future.result()

# Print results
print(f"✓ Dataset works! {result['total_samples']} total patches")
print(f"  Grid: {result['metadata']}")
for s in result['sample_tests']:
    print(f"  Sample {s['idx']}: features={s['features_shape']}, labels={s['labels_shape']}")

✓ DataLoader works! 1.9 patches/sec
✓ Dataset works! 6149 total patches
  Grid: {'n_time': 43, 'n_y': 13, 'n_x': 11, 'total_patches': 6149, 'dims': {'band': 6, 'time': 43, 'y': 6441, 'x': 5469}}
  Sample 0: features=(6, 512, 512), labels=(512, 512)
  Sample 1: features=(6, 512, 512), labels=(512, 512)
  Sample 2: features=(6, 512, 512), labels=(512, 512)


In [99]:
def test_dataloader_on_worker(zarr_path: str, batch_size: int = 4):
    """Test full DataLoader pipeline on worker."""
    from torch.utils.data import DataLoader
    import time
    
    ds = RemoteZarrDataset(zarr_path, PATCH_SIZE, 0, 1)
    
    # num_workers=0 required inside Dask workers (daemon process limitation)
    loader = DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=0)
    
    start = time.time()
    for i, (feat, lab) in enumerate(loader):
        if i >= 3:
            break
    elapsed = time.time() - start
    
    return {
        'batches_tested': 3,
        'batch_shape': tuple(feat.shape),
        'time_seconds': elapsed,
        'patches_per_second': (3 * batch_size) / elapsed,
    }

result = client.submit(test_dataloader_on_worker, S3_OUTPUT_PATH).result()
print(f"✓ DataLoader works! {result['patches_per_second']:.1f} patches/sec")

✓ DataLoader works! 3.8 patches/sec


In [100]:
from threading import Thread
from queue import Queue
import torch

class ThreadedPrefetcher:
    """
    Prefetch batches using threads instead of processes.
    Works inside Dask daemon workers.
    """
    
    def __init__(self, dataset, batch_size: int, shuffle: bool = True, prefetch: int = 2):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.prefetch = prefetch
        self.queue = Queue(maxsize=prefetch)
        self._stop = False
    
    def _producer(self, indices):
        """Background thread that fetches batches."""
        try:
            for batch_start in range(0, len(indices), self.batch_size):
                if self._stop:
                    break
                
                batch_indices = indices[batch_start:batch_start + self.batch_size]
                features_list = []
                labels_list = []
                
                for idx in batch_indices:
                    feat, lab = self.dataset[idx]
                    features_list.append(feat)
                    labels_list.append(lab)
                
                batch = (
                    torch.stack(features_list),
                    torch.stack(labels_list)
                )
                self.queue.put(batch)
            
            self.queue.put(None)  # Signal end
        except Exception as e:
            self.queue.put(e)
    
    def __iter__(self):
        import random
        
        indices = list(range(len(self.dataset)))
        if self.shuffle:
            random.shuffle(indices)
        
        # Start producer thread
        self._stop = False
        thread = Thread(target=self._producer, args=(indices,), daemon=True)
        thread.start()
        
        # Yield batches
        while True:
            item = self.queue.get()
            if item is None:
                break
            if isinstance(item, Exception):
                raise item
            yield item
        
        thread.join()
    
    def __len__(self):
        return (len(self.dataset) + self.batch_size - 1) // self.batch_size


# Test it
def test_threaded_loader(zarr_path: str, batch_size: int = 4):
    """Test threaded prefetcher on worker."""
    import time
    
    ds = RemoteZarrDataset(zarr_path, PATCH_SIZE, 0, 1)
    loader = ThreadedPrefetcher(ds, batch_size=batch_size, shuffle=True, prefetch=3)
    
    start = time.time()
    for i, (feat, lab) in enumerate(loader):
        if i >= 3:
            break
    elapsed = time.time() - start
    
    return {
        'batch_shape': tuple(feat.shape),
        'time_seconds': elapsed,
        'patches_per_second': (3 * batch_size) / elapsed,
    }

result = client.submit(test_threaded_loader, S3_OUTPUT_PATH).result()
print(f"✓ Threaded loader: {result['patches_per_second']:.1f} patches/sec")

✓ Threaded loader: 4.9 patches/sec


In [101]:
from concurrent.futures import ThreadPoolExecutor
import torch

class AsyncBatchLoader:
    """
    Uses thread pool for concurrent S3 fetches.
    Better throughput than sequential loading.
    """
    
    def __init__(
        self, 
        dataset, 
        batch_size: int, 
        shuffle: bool = True,
        num_threads: int = 4
    ):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.num_threads = num_threads
    
    def __iter__(self):
        import random
        
        indices = list(range(len(self.dataset)))
        if self.shuffle:
            random.shuffle(indices)
        
        with ThreadPoolExecutor(max_workers=self.num_threads) as executor:
            for batch_start in range(0, len(indices), self.batch_size):
                batch_indices = indices[batch_start:batch_start + self.batch_size]
                
                # Fetch all items in batch concurrently
                futures = [
                    executor.submit(self.dataset.__getitem__, idx) 
                    for idx in batch_indices
                ]
                
                results = [f.result() for f in futures]
                features = torch.stack([r[0] for r in results])
                labels = torch.stack([r[1] for r in results])
                
                yield features, labels
    
    def __len__(self):
        return (len(self.dataset) + self.batch_size - 1) // self.batch_size


# Test it
def test_async_loader(zarr_path: str, batch_size: int = 4):
    import time
    
    ds = RemoteZarrDataset(zarr_path, PATCH_SIZE, 0, 1)
    loader = AsyncBatchLoader(ds, batch_size=batch_size, num_threads=4)
    
    start = time.time()
    for i, (feat, lab) in enumerate(loader):
        if i >= 3:
            break
    elapsed = time.time() - start
    
    return {
        'batch_shape': tuple(feat.shape),
        'time_seconds': elapsed,
        'patches_per_second': (3 * batch_size) / elapsed,
    }

result = client.submit(test_async_loader, S3_OUTPUT_PATH).result()
print(f"✓ Async loader: {result['patches_per_second']:.1f} patches/sec")

✓ Async loader: 7.4 patches/sec


# Training Experiments
Tests to use the working dataset module to fine tune a model

In [ ]:
gateway = Gateway()
options = gateway.cluster_options()
options.cuda_worker = True          # important for GPU training on EASI

cluster = gateway.new_cluster(cluster_options=options)
NUM_WORKERS = 4
cluster.scale(NUM_WORKERS)
client = cluster.get_client()

print("Dashboard:", client.dashboard_link)

# S3 access for all workers
from datacube.utils.aws import configure_s3_access
configure_s3_access(aws_unsigned=False, requester_pays=True, client=client)

# Sanity check: open Zarr once from the client
import fsspec, xarray as xr
s3_mapper = fsspec.get_mapper(S3_OUTPUT_PATH)
xr.open_zarr(s3_mapper, consolidated=True)  # should succeed quickly

In [24]:
def train_ddp_rank(
    rank: int,
    world_size: int,
    master_addr: str,
    master_port: int,
    zarr_path: str,
    patch_size: int,
    num_epochs: int = 1,
    batch_size: int = 4,
    lr: float = 1e-4,
):
    import os
    import torch
    import torch.nn as nn
    import torch.optim as optim
    import torch.distributed as dist
    from torch.nn.parallel import DistributedDataParallel as DDP

    # --- DDP init ---
    os.environ['MASTER_ADDR'] = master_addr
    os.environ['MASTER_PORT'] = str(master_port)
    os.environ['RANK'] = str(rank)
    os.environ['WORLD_SIZE'] = str(world_size)

    backend = 'nccl' if torch.cuda.is_available() else 'gloo'
    dist.init_process_group(backend=backend, init_method='env://')

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    if device.type == 'cuda':
        torch.cuda.set_device(0)  # each worker pod sees a single GPU as cuda:0
        print("cuda")

    # --- Dataset shard for this rank ---
    dataset = RemoteZarrDataset(
        zarr_path=zarr_path,
        patch_size=patch_size,
        worker_rank=rank,
        world_size=world_size,
        feature_var='features',
        label_var='labels',
    )

    # *** IMPORTANT: force lazy init before using _ds ***
    dataset._ensure_initialized()
    xr_ds = dataset._ds

    # --- Model: REPLACE this with your foundation model loader ---
    # n_channels from 'band' dimension of features
    n_channels = xr_ds['features'].sizes['band']

    # Ideally set this from product metadata; keeping 256 as a placeholder
    num_classes = 256

    class SmallUNetLike(nn.Module):
        def __init__(self, in_ch, out_ch):
            super().__init__()
            self.encoder = nn.Sequential(
                nn.Conv2d(in_ch, 32, 3, padding=1),
                nn.ReLU(),
                nn.Conv2d(32, 64, 3, padding=1),
                nn.ReLU(),
            )
            self.decoder = nn.Sequential(
                nn.Conv2d(64, 64, 3, padding=1),
                nn.ReLU(),
                nn.Conv2d(64, out_ch, 1),
            )
        def forward(self, x):
            x = self.encoder(x)
            x = self.decoder(x)
            return x

    model = SmallUNetLike(n_channels, num_classes).to(device)

    ddp_model = DDP(model, device_ids=[0] if device.type == 'cuda' else None)

    criterion = nn.CrossEntropyLoss(ignore_index=255)
    optimizer = optim.AdamW(ddp_model.parameters(), lr=lr)
    scaler = torch.cuda.amp.GradScaler(enabled=(device.type == 'cuda'))

    ddp_model.train()
    last_loss = None

    for epoch in range(num_epochs):
        for X, y in dataset.iter_batches(batch_size=batch_size, shuffle=True):
            X = X.to(device, non_blocking=True)
            y = y.to(device, non_blocking=True)

            optimizer.zero_grad()
            with torch.cuda.amp.autocast(enabled=(device.type == 'cuda')):
                logits = ddp_model(X)
                loss = criterion(logits, y)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            last_loss = loss.detach()

    if last_loss is None:
        last_loss = torch.tensor(0.0, device=device)
    else:
        last_loss = last_loss.to(device)

    dist.all_reduce(last_loss, op=dist.ReduceOp.SUM)
    last_loss = (last_loss / world_size).item()

    if rank == 0:
        # torch.save(ddp_model.module.state_dict(), "ddp_model_final.pt")
        pass

    dist.destroy_process_group()
    return last_loss

In [25]:
# client.restart()
ds = RemoteZarrDataset(
    zarr_path=S3_OUTPUT_PATH,
    patch_size=PATCH_SIZE,
    worker_rank=0,
    world_size=1,
    feature_var='features',
    label_var='labels',
)

x, y = ds[0]
print("x shape:", x.shape, "y shape:", y.shape)
print("x NaNs:", torch.isnan(x).any().item())
print("y unique values:", torch.unique(y)[:20])  # first 20 classes
print("y min / max:", y.min().item(), y.max().item())

x shape: torch.Size([6, 224, 224]) y shape: torch.Size([224, 224])
x NaNs: False
y unique values: tensor([111, 112, 216, 220])
y min / max: 111 220


In [26]:
info = client.scheduler_info()
print("Got scheduler info")

worker_addrs = list(info['workers'].keys())
print("Workers:", worker_addrs)

world_size = len(worker_addrs)
print("World size:", world_size, " expected:", NUM_WORKERS)

assert world_size == NUM_WORKERS, (world_size, NUM_WORKERS)
print("World size assertion passed")

master_worker = worker_addrs[0]
master_host = info['workers'][master_worker].get('host', None)
if master_host is None:
    master_host = master_worker.split('://')[-1].split(':')[0]
master_port = 23456
print(f"Master is {master_worker} ({master_host}:{master_port})")

futures = []
print("Submitting DDP training tasks...")
for rank, worker in enumerate(worker_addrs):
    print(f"  Submitting rank {rank} to worker {worker}")
    fut = client.submit(
        train_ddp_rank,
        rank,
        world_size,
        master_host,
        master_port,
        S3_OUTPUT_PATH,
        PATCH_SIZE,
        num_epochs=1,
        batch_size=4,
        lr=1e-4,
        workers=[worker],
        pure=False,
    )
    futures.append(fut)

losses = client.gather(futures)
print("Final averaged loss per rank:", losses)

Got scheduler info
Workers: ['tls://10.0.38.3:40523', 'tls://10.0.38.4:38895', 'tls://10.0.38.5:35171', 'tls://10.0.38.6:46131']
World size: 4  expected: 4
World size assertion passed
Master is tls://10.0.38.3:40523 (10.0.38.3:23456)
Submitting DDP training tasks...
  Submitting rank 0 to worker tls://10.0.38.3:40523
  Submitting rank 1 to worker tls://10.0.38.4:38895
  Submitting rank 2 to worker tls://10.0.38.5:35171
  Submitting rank 3 to worker tls://10.0.38.6:46131


KeyboardInterrupt: 

In [63]:
from dask.distributed import wait
import torch
import io
from typing import Dict, List


class PrithviDistributedTrainer:
    """
    Orchestrates distributed Prithvi training across Dask workers.
    Lightweight - only submits jobs and collects results.
    """
    
    def __init__(self, client):
        self.client = client
        self.workers = list(client.scheduler_info()['workers'].keys())
        self.world_size = len(self.workers)
        print(f"Trainer initialized with {self.world_size} workers")
    
    def train(
        self,
        zarr_path: str,
        num_classes: int = 2,
        epochs: int = 10,
        batch_size: int = 4,
        learning_rate: float = 1e-4,
        patch_size: int = PATCH_SIZE,
        freeze_backbone_epochs: int = 2,
    ) -> Dict:
        """
        Launch distributed training.
        
        Args:
            zarr_path: S3 path to training Zarr
            num_classes: Number of output classes
            epochs: Training epochs
            batch_size: Batch size per worker
            learning_rate: Learning rate
            patch_size: Input patch size
            freeze_backbone_epochs: Epochs to keep backbone frozen
        
        Returns:
            Dictionary with trained model and metrics
        """
        print(f"\n{'='*60}")
        print("Starting Distributed Prithvi Training")
        print(f"{'='*60}")
        print(f"Data: {zarr_path}")
        print(f"Workers: {self.world_size}")
        print(f"Epochs: {epochs}")
        print(f"Batch size: {batch_size} per worker")
        print(f"Classes: {num_classes}")
        
        # Upload code to workers
        print("\nUploading code to workers...")
        self.client.upload_file('distributed_dataset.py')
        self.client.upload_file('prithvi_model.py')
        self.client.upload_file('train_prithvi.py')
        
        # Submit training jobs
        print(f"\nSubmitting training jobs to {self.world_size} workers...")
        futures = []
        
        for rank, worker_addr in enumerate(self.workers):
            future = self.client.submit(
                train_prithvi_on_worker,
                zarr_path=zarr_path,
                model_state=None,  # Start fresh
                worker_rank=rank,
                world_size=self.world_size,
                num_classes=num_classes,
                epochs=epochs,
                batch_size=batch_size,
                learning_rate=learning_rate,
                patch_size=patch_size,
                freeze_backbone_epochs=freeze_backbone_epochs,
                workers=[worker_addr],
                pure=False,
                key=f'prithvi-train-{rank}'
            )
            futures.append(future)
        
        print(f"\nTraining in progress...")
        print(f"Dashboard: {self.client.dashboard_link}")
        
        # Wait for all workers
        wait(futures)
        
        # Collect results
        print("\nCollecting results...")
        results = self.client.gather(futures)
        
        # Average model weights
        print("Averaging model weights (FedAvg)...")
        averaged_state = self._average_weights([r['model_state'] for r in results])
        
        # Combine metrics
        combined_metrics = self._combine_metrics(results)
        
        print(f"\n{'='*60}")
        print("Training Complete!")
        print(f"{'='*60}")
        print(f"Total samples processed: {combined_metrics['total_samples']}")
        print(f"Final average loss: {combined_metrics['final_loss']:.4f}")
        print(f"Final water IoU: {combined_metrics['final_water_iou']:.4f}")
        
        return {
            'model_state': averaged_state,
            'metrics': combined_metrics,
            'worker_results': results,
        }
    
    def _average_weights(self, state_bytes_list: List[bytes]) -> Dict:
        """Average model weights from all workers."""
        states = [
            torch.load(io.BytesIO(b), map_location='cpu') 
            for b in state_bytes_list
        ]
        
        averaged = {}
        for key in states[0].keys():
            tensors = [s[key].float() for s in states]
            averaged[key] = sum(tensors) / len(tensors)
        
        return averaged
    
    def _combine_metrics(self, results: List[Dict]) -> Dict:
        """Combine metrics from all workers."""
        total_samples = sum(r['metrics']['samples_processed'] for r in results)
        
        # Average the per-epoch metrics
        all_losses = []
        all_ious = []
        for r in results:
            all_losses.extend(r['metrics']['train_loss'])
            all_ious.extend(r['metrics']['water_iou'])
        
        return {
            'total_samples': total_samples,
            'final_loss': sum(r['final_loss'] for r in results) / len(results),
            'final_water_iou': sum(r['final_water_iou'] for r in results) / len(results),
            'loss_history': all_losses,
            'water_iou_history': all_ious,
            'per_worker': [
                {
                    'rank': r['worker_rank'],
                    'loss': r['final_loss'],
                    'water_iou': r['final_water_iou'],
                }
                for r in results
            ]
        }


# Import the training function for the workers
from train_prithvi import train_prithvi_on_worker

In [82]:
client.wait_for_workers(NUM_WORKERS)
print(f"✓ {NUM_WORKERS} workers ready")

# Verify data is accessible
def check_data(path):
    ds = RemoteZarrDataset(path, 224, 0, 1)
    sample = ds[0]
    return {
        'num_patches': len(ds),
        'feature_shape': tuple(sample[0].shape),
        'label_shape': tuple(sample[1].shape),
        'unique_labels': sample[1].unique().tolist(),
    }

result = client.submit(check_data, S3_OUTPUT_PATH).result()
print(f"✓ Data check: {result}")

✓ 4 workers ready
✓ Data check: {'num_patches': 858, 'feature_shape': (6, 224, 224), 'label_shape': (224, 224), 'unique_labels': [255]}


In [78]:
# ============================================================
# CELL 2: Configure training
# ============================================================

# Training configuration
CONFIG = {
    'zarr_path': S3_OUTPUT_PATH,
    'num_classes': 2,           # water, not-water
    'epochs': 10,
    'batch_size': 4,            # Per worker
    'learning_rate': 1e-4,
    'patch_size': PATCH_SIZE,
    'freeze_backbone_epochs': 2,  # Train decoder first, then fine-tune all
}

print("Training configuration:")
for k, v in CONFIG.items():
    print(f"  {k}: {v}")

Training configuration:
  zarr_path: s3://easihub-csiro-user-scratch/AROAWO7MSC2T2SINGSYOQ:csiro-csiro-aad_gre555@csiro.au/traning_test_project/training_dataset_v1.zarr
  num_classes: 2
  epochs: 10
  batch_size: 4
  learning_rate: 0.0001
  patch_size: 512
  freeze_backbone_epochs: 2


In [84]:
# Restart workers to clear all cached imports
print("Restarting workers...")
client.restart()
client.wait_for_workers(NUM_WORKERS)
print(f"✓ Workers restarted")

# Upload fresh code
client.upload_file('distributed_dataset.py')
client.upload_file('prithvi_model.py')
client.upload_file('train_prithvi.py')
print("✓ Code uploaded")

# Now test
def quick_test(zarr_path):
    from prithvi_model import PrithviWaterSegmentation
    from distributed_dataset import RemoteZarrDataset
    import torch
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    ds = RemoteZarrDataset(zarr_path, PATCH_SIZE, 0, 1)
    feat, lab = ds[0]
    
    model = PrithviWaterSegmentation(num_classes=2, freeze_backbone=True).to(device)
    model.eval()
    
    with torch.no_grad():
        x = feat.unsqueeze(0).to(device)
        out = model(x)
    
    return {'input': list(x.shape), 'output': list(out.shape)}

result = client.submit(quick_test, S3_OUTPUT_PATH).result()
print(f"✓ Test passed: {result}")

Restarting workers...
✓ Workers restarted
✓ Code uploaded


RuntimeError: shape '[1, 768, 32, 32]' is invalid for input of size 785664

In [88]:
def test_model_inline(zarr_path: str):
    """
    Complete test with everything defined inline.
    No external file dependencies except terratorch and distributed_dataset.
    """
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    from terratorch.registry import BACKBONE_REGISTRY
    
    # Import dataset (this one should work since it was tested)
    from distributed_dataset import RemoteZarrDataset
    
    print("=== Starting inline model test ===")
    
    # ============================================
    # STEP 1: Check what the backbone outputs
    # ============================================
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Device: {device}")
    
    # Load a sample first to get dimensions
    ds = RemoteZarrDataset(zarr_path, 224, 0, 1)
    features, labels = ds[0]
    print(f"Features shape: {features.shape}")  # Should be (C, H, W)
    print(f"Labels shape: {labels.shape}")      # Should be (H, W)
    
    C, H, W = features.shape
    print(f"Image size: {H}x{W}, Channels: {C}")
    
    # ============================================
    # STEP 2: Load backbone and check output
    # ============================================
    print("\nLoading Prithvi backbone...")
    backbone = BACKBONE_REGISTRY.build(
        'terratorch_prithvi_eo_v1_100',
        pretrained=True
    )
    backbone = backbone.to(device)
    backbone.eval()
    
    # Test backbone output
    x = features.unsqueeze(0).to(device)       # (1, C, H, W)
    x_temporal = x.unsqueeze(2)                 # (1, C, 1, H, W)
    print(f"Backbone input: {x_temporal.shape}")
    
    with torch.no_grad():
        backbone_out = backbone(x_temporal)
    
    # Analyze output
    if isinstance(backbone_out, list):
        print(f"Backbone output: list of {len(backbone_out)} tensors")
        tokens = backbone_out[-1]
    else:
        tokens = backbone_out
    
    print(f"Tokens shape: {tokens.shape}")  # (B, N, C)
    B, N, embed_dim = tokens.shape
    
    # ============================================
    # STEP 3: Figure out spatial dimensions
    # ============================================
    patch_size = 16  # Prithvi uses 16x16 patches
    h_patches = H // patch_size
    w_patches = W // patch_size
    expected_spatial = h_patches * w_patches
    
    print(f"\nPatch calculation:")
    print(f"  Image: {H}x{W}, Patch size: {patch_size}")
    print(f"  Expected spatial tokens: {h_patches}x{w_patches} = {expected_spatial}")
    print(f"  Actual tokens: {N}")
    print(f"  Difference: {N - expected_spatial} (likely CLS token)")
    
    # Remove CLS token if present
    if N == expected_spatial + 1:
        print(f"  Removing CLS token (first token)")
        tokens = tokens[:, 1:, :]
    elif N > expected_spatial + 1:
        extra = N - expected_spatial
        print(f"  Removing {extra} special tokens")
        tokens = tokens[:, extra:, :]
    
    print(f"Tokens after CLS removal: {tokens.shape}")
    
    # ============================================
    # STEP 4: Reshape to feature map
    # ============================================
    # (B, N, C) -> (B, C, H, W)
    feat_map = tokens.transpose(1, 2).reshape(B, embed_dim, h_patches, w_patches)
    print(f"Feature map: {feat_map.shape}")
    
    # ============================================
    # STEP 5: Build and test decoder
    # ============================================
    print("\nBuilding decoder...")
    
    decoder = nn.Sequential(
        nn.ConvTranspose2d(embed_dim, 256, kernel_size=2, stride=2),
        nn.BatchNorm2d(256),
        nn.ReLU(inplace=True),
        nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2),
        nn.BatchNorm2d(128),
        nn.ReLU(inplace=True),
        nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2),
        nn.BatchNorm2d(64),
        nn.ReLU(inplace=True),
    ).to(device)
    
    seg_head = nn.Conv2d(64, 2, kernel_size=1).to(device)
    
    with torch.no_grad():
        decoded = decoder(feat_map)
        print(f"Decoded: {decoded.shape}")
        
        logits = seg_head(decoded)
        print(f"Logits (pre-upsample): {logits.shape}")
        
        logits = F.interpolate(logits, size=(H, W), mode='bilinear', align_corners=False)
        print(f"Logits (final): {logits.shape}")
        
        preds = logits.argmax(dim=1)
        print(f"Predictions: {preds.shape}")
    
    # ============================================
    # STEP 6: Check predictions
    # ============================================
    water_pred = (preds == 1).float().mean().item() * 100
    water_actual = (labels == 1).float().mean().item() * 100
    
    print(f"\nResults:")
    print(f"  Predicted water: {water_pred:.1f}%")
    print(f"  Actual water: {water_actual:.1f}%")
    
    return {
        'success': True,
        'input_shape': list(x.shape),
        'output_shape': list(logits.shape),
        'tokens_shape': [B, N, embed_dim],
        'feature_map_shape': list(feat_map.shape),
        'water_pred_pct': water_pred,
        'water_actual_pct': water_actual,
    }


# Run the test
result = client.submit(test_model_inline, S3_OUTPUT_PATH).result()

print("\n" + "="*50)
print("TEST RESULTS")
print("="*50)
for k, v in result.items():
    print(f"  {k}: {v}")


TEST RESULTS
  success: True
  input_shape: [1, 6, 224, 224]
  output_shape: [1, 2, 224, 224]
  tokens_shape: [1, 197, 768]
  feature_map_shape: [1, 768, 14, 14]
  water_pred_pct: 93.78387928009033
  water_actual_pct: 0.0


In [83]:
# Test the fixed model
def test_model_forward(zarr_path: str):
    """Test model forward pass on worker."""
    from distributed_dataset import RemoteZarrDataset
    from prithvi_model import PrithviWaterSegmentation
    import torch
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Load one sample
    ds = RemoteZarrDataset(zarr_path, 224, 0, 1)
    features, labels = ds[0]
    print(f"Input: {features.shape}, Labels: {labels.shape}")
    
    # Create model
    model = PrithviWaterSegmentation(num_classes=2, freeze_backbone=True)
    model = model.to(device)
    model.eval()
    
    # Forward pass
    with torch.no_grad():
        x = features.unsqueeze(0).to(device)
        print(f"Model input: {x.shape}")
        
        logits = model(x)
        print(f"Model output: {logits.shape}")
        
        preds = logits.argmax(dim=1)
        print(f"Predictions: {preds.shape}")
        
        # Check prediction distribution
        water_pct = (preds == 1).float().mean().item() * 100
        print(f"Predicted water: {water_pct:.1f}%")
    
    return {
        'input_shape': list(x.shape),
        'output_shape': list(logits.shape),
        'pred_shape': list(preds.shape),
        'water_pct': water_pct,
        'success': True
    }

result = client.submit(test_model_forward, S3_OUTPUT_PATH).result()
print(f"\n✓ Model forward pass successful!")
print(f"  Input:  {result['input_shape']}")
print(f"  Output: {result['output_shape']}")
print(f"  Preds:  {result['pred_shape']}")

RuntimeError: shape '[1, 768, 14, 14]' is invalid for input of size 151296

In [68]:
def debug_backbone_output(zarr_path: str):
    """Debug what the Prithvi backbone actually outputs."""
    from distributed_dataset import RemoteZarrDataset
    import torch
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Load backbone only
    from terratorch.registry import BACKBONE_REGISTRY
    backbone = BACKBONE_REGISTRY.build('terratorch_prithvi_eo_v1_100', pretrained=True)
    backbone = backbone.to(device)
    backbone.eval()
    
    # Get sample
    ds = RemoteZarrDataset(zarr_path, PATCH_SIZE, 0, 1)
    features, _ = ds[0]
    
    # Prepare input
    x = features.unsqueeze(0).unsqueeze(2).to(device)  # (1, C, 1, H, W)
    print(f"Backbone input shape: {x.shape}")
    
    # Forward
    with torch.no_grad():
        out = backbone(x)
    
    # Analyze output
    result = {'input_shape': tuple(x.shape)}
    
    if isinstance(out, torch.Tensor):
        result['output_type'] = 'tensor'
        result['output_shape'] = tuple(out.shape)
    elif isinstance(out, dict):
        result['output_type'] = 'dict'
        result['keys'] = list(out.keys())
        result['shapes'] = {k: tuple(v.shape) if isinstance(v, torch.Tensor) else type(v).__name__ 
                           for k, v in out.items()}
    elif isinstance(out, (list, tuple)):
        result['output_type'] = 'list'
        result['length'] = len(out)
        result['shapes'] = [tuple(o.shape) if isinstance(o, torch.Tensor) else type(o).__name__ 
                           for o in out]
    
    return result

# Debug
debug_result = client.submit(debug_backbone_output, S3_OUTPUT_PATH).result()
print("Backbone output structure:")
for k, v in debug_result.items():
    print(f"  {k}: {v}")

Backbone output structure:
  input_shape: (1, 6, 1, 512, 512)
  output_type: list
  length: 12
  shapes: [(1, 1025, 768), (1, 1025, 768), (1, 1025, 768), (1, 1025, 768), (1, 1025, 768), (1, 1025, 768), (1, 1025, 768), (1, 1025, 768), (1, 1025, 768), (1, 1025, 768), (1, 1025, 768), (1, 1025, 768)]


In [66]:
# ============================================================
# CELL 3: Run training
# ============================================================

# Create trainer
trainer = PrithviDistributedTrainer(client)

# Run distributed training
results = trainer.train(**CONFIG)

# Save model state for later
trained_model_state = results['model_state']
training_metrics = results['metrics']

Trainer initialized with 4 workers

Starting Distributed Prithvi Training
Data: s3://easihub-csiro-user-scratch/AROAWO7MSC2T2SINGSYOQ:csiro-csiro-aad_gre555@csiro.au/traning_test_project/training_dataset_v1.zarr
Workers: 4
Epochs: 10
Batch size: 4 per worker
Classes: 2

Uploading code to workers...

Submitting training jobs to 4 workers...

Training in progress...
Dashboard: https://hub.csiro.easi-eo.solutions/services/dask-gateway/clusters/easihub.cf1478225bb14fceae84e4b8d19c101c/status



RuntimeError: shape '[4, 768, 32, 32]' is invalid for input of size 3148800

In [60]:
# ============================================================
# CELL 4: Save trained model to S3
# ============================================================

def save_to_s3(model_state: bytes, metrics: dict, s3_path: str):
    """Save model and metrics to S3 - runs on worker."""
    import fsspec
    import json
    import torch
    import io
    
    # Save model
    model_path = f"{s3_path}/prithvi_water_model.pt"
    with fsspec.open(model_path, 'wb') as f:
        buffer = io.BytesIO()
        torch.save(model_state, buffer)
        f.write(buffer.getvalue())
    
    # Save metrics
    metrics_path = f"{s3_path}/training_metrics.json"
    # Convert non-serializable items
    metrics_serializable = {
        k: v if not isinstance(v, (list,)) or not v or not hasattr(v[0], 'item') else [x for x in v]
        for k, v in metrics.items()
    }
    with fsspec.open(metrics_path, 'w') as f:
        json.dump(metrics_serializable, f, indent=2, default=str)
    
    return {'model_path': model_path, 'metrics_path': metrics_path}

# Save via worker
import io
buffer = io.BytesIO()
torch.save(trained_model_state, buffer)
model_bytes = buffer.getvalue()

save_path = f"s3://{bucket}/{userid}/{project_name}/models"
result = client.submit(save_to_s3, model_bytes, training_metrics, save_path).result()
print(f"✓ Model saved to: {result['model_path']}")
print(f"✓ Metrics saved to: {result['metrics_path']}")

NameError: name 'trained_model_state' is not defined

In [27]:
client.close()
cluster.shutdown()